In [4]:
#!/usr/bin/env python

import numpy as np
import sys
import re
import os

class world:

	def __init__(self):

		# Reward on current state
		self.rmatrix = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0])

		# For all states on all 4 actions probability to reach other state
		# Actions in order: W, E, N, S

		# 3D array: S X A X S
		self.M = np.zeros([9, 4, 9])

		self.gamma = 0


	#randomly fill values in M
	def populateM(self, S, A):

		for i in range(0, S):

			seed = np.random.rand(1)
			p = (1 - seed)/(S-1)
	
			for a in range(0, A):

				for j in range(0, S):

					if i == j:
					    self.M[i, a, j] = seed

					else:
					    self.M[i, a, j] = p 

	
	def U(self, Vin):

		#Vin is S X 1 matrix

		S = len(Vin)
		Vorg = np.copy(Vin)
		Vnew = np.copy(Vin)

		for i in range(0, S):	# for all rows Vin, for all states
			temp = []			

			for a in range(0, 4):		# check max in all actions

				p = 0
				for j in range(0, S):
					p = p + self.M[i, a, j]*Vorg[j] 

				
				t = self.rmatrix[i] + (self.gamma)*(p)
				temp.append(t)

			#print temp
			Vnew[i] = max(temp)

		return Vnew

	def policy_evaluation(self, P, V):

		S = len(P)
		Vnew = np.copy(V)


		while 1:
			old_V = Vnew
			#V_init = w.U(V_init)

			for i in range(0, S):			
				sum = 0
				for j in range(0, S):
					sum = sum + self.M[i, int(P[i]), j]*V[j]

				
				Vnew[i] = self.rmatrix[i] + (self.gamma)*(sum)


			if np.array_equal(old_V, Vnew) :
			    break
		
		return Vnew

#Convert 1D V to 2D V
def getV(V_init):

	V = np.zeros([3,3])
	ind = 0

	for i in range(0, 3):
		for j in range(0, 3):
			V[i, j] = float(V_init[ind])
			ind += 1

	return V

#Convert 2D P to 1D
def getP(P):

	N = len(P[0])
	Pnew = np.empty([N*N])
	ind = 0

	for i in range(0, N):
		for j in range(0, N):
			Pnew[ind] = P[i, j]
			ind += 1

	return Pnew

#Get policy from value function
def getPolicy(V):

	N = len(V[0])
	Policy = np.empty([N,N],dtype=int)

	for i in range(0, N):
		for j in range(0, N):
			max = 0

			# Note Im not comparing i,j value
			# ie no stay option

			# look at all four neighbours
			if i-1 >=0:						#Up
				if V[i-1, j] > max:
					max = V[i-1, j]
					Policy[i, j] = 0

			if i+1 < N:						# Down
				if V[i+1, j] > max:
					max = V[i+1, j]
					Policy[i, j] = 1
			
			if j+1 < N:						# Right
				if V[i, j+1] > max:
					max = V[i, j+1]
					Policy[i, j] = 2

			if j-1 >=0:						# Left
				if V[i, j-1] > max:
					max = V[i, j-1]
					Policy[i, j] = 3

	return Policy

#For printing directions
def printPolicy(P):

	N = len(P[0])
	D = np.empty([N, N], dtype=str)

	for i in range(0, N):
		for j in range(0, N):

			if P[i, j] == 0:		#Up
				D[i, j] = '^'
			if P[i, j] == 1:		#Down
				D[i, j] = 'd'
			if P[i, j] == 2:		#Right
				D[i, j] = '>'
			if P[i, j] == 3:		#Left
				D[i, j] = '<'

	print(D)
				




if __name__ == '__main__':


	print('Enter value of gamma')
	gamma = float(input())
	
	w = world()
	w.gamma = gamma

	# stay in state value
	S = 9
	A = 4

	V_init = np.zeros([S])
	w.populateM(S, A)

	updates1 = 0

	while 1:
		old_V_int = V_init
		V_init = w.U(V_init)
		updates1 += 1

		if np.array_equal(old_V_int, V_init) :
			break
		
	V_opt = getV(V_init)
	print('Value Iteration result')
	print ('V optimal-')
	print (V_opt)
	print ('\nOptimal policy')
	printPolicy(getPolicy(V_opt))
	print ('Total updates =',updates1)

	print ('\nPolicy Iteration results-')

	#Arbitrary policy
	policy = np.zeros([S])
	#Arbitrary V
	Vnew = V_init

	updates2 = 0

	while 1:

		policy_old = policy

		#Policy evaluation
		Vnew = w.policy_evaluation(policy, Vnew)

		#Policy Improvement
		ptemp = getPolicy(getV(Vnew))	#returns 2D

		policy = getP(ptemp)			#save as 1D

		updates2 += 1
		if np.array_equal(policy, policy_old):
			break	

	print('\nV optimal')
	print (getV(Vnew))
	print ('Optimal policy')
	print (printPolicy(ptemp))
	print ('Total updates =',updates2)


Enter value of gamma
0.9
Value Iteration result
V optimal-
[[0.70078329 0.24866905 2.82187146]
 [0.61830574 0.52234926 0.59964066]
 [0.63856342 0.73155129 0.45047224]]

Optimal policy
[['d' '>' 'd']
 ['^' 'd' '^']
 ['>' '<' '<']]
Total updates = 329

Policy Iteration results-

V optimal
[[0.70078329 0.24866905 2.82187146]
 [0.61830574 0.52234926 0.59964066]
 [0.63856342 0.73155129 0.45047224]]
Optimal policy
[['d' '>' 'd']
 ['^' 'd' '^']
 ['>' '<' '<']]
None
Total updates = 2
